In [76]:
import pandas as pd
import numpy as np


In [77]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
energy_efficiency = fetch_ucirepo(id=242) 
  
# data (as pandas dataframes) 
X = energy_efficiency.data.features 
y = energy_efficiency.data.targets 
  
# metadata 
print(energy_efficiency.metadata) 
  
# variable information 
print(energy_efficiency.variables) 

{'uci_id': 242, 'name': 'Energy Efficiency', 'repository_url': 'https://archive.ics.uci.edu/dataset/242/energy+efficiency', 'data_url': 'https://archive.ics.uci.edu/static/public/242/data.csv', 'abstract': 'This study looked into assessing the heating load and cooling load requirements of buildings (that is, energy efficiency) as a function of building parameters.', 'area': 'Computer Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 768, 'num_features': 8, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Y1', 'Y2'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2012, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C51307', 'creators': ['Athanasios Tsanas', 'Angeliki Xifara'], 'intro_paper': {'ID': 379, 'type': 'NATIVE', 'title': 'Accurate quantitative estimation of energy performance of residential buildings using statistical machine 

In [78]:
def train_valid_test_split(X, y):
    ''' Split the data into train, validation and test datasets using pandas'''
    train=int(X.shape[0]*0.7)
    valid=int(X.shape[0]*0.1)
    test=int(X.shape[0]*0.2)
    print("Data amounts to training data {}, validation data {} and testing data {}. ".format(train, valid, test))

    X_df=pd.DataFrame(X, dtype=float)
    X_df.insert(0, 'W0', 1)
    X_train=X_df.iloc[0:train,:].values
    X_valid=X_df.iloc[train:train+valid,:].values
    X_test=X_df.iloc[train+valid:,:].values
    y_df=pd.DataFrame(y, dtype=float)
    y_train=y_df.iloc[0:train,:].values
    y_valid=y_df.iloc[train:train+valid,:].values
    y_test=y_df.iloc[train+valid:,:].values
    
    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [79]:
def w_hat_lin_calc(X_train, y_train):
    w_hat=np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
    return w_hat

def w_hat_lin_calc_pseudo(X_train, y_train):
    w_hat=np.linalg.pinv(X_train.T @ X_train) @ X_train.T @ y_train
    return w_hat

def w_hat_ridge_calc(X_train, y_train,lamb):
    n_features=X_train.shape[1]
    w_hat=np.linalg.inv(X_train.T @ X_train + lamb*np.eye(n_features)) @ X_train.T @ y_train
    return w_hat

def predict(X, w_hat):
    y_predict=X @ w_hat
    return y_predict

def compare(y_test, y_predict,n):
    mae_i = np.mean(np.abs(y_test - y_predict))
    mse_i = np.mean((y_test - y_predict) ** 2)
    rmse_i = np.sqrt(mse_i)
    print("For the Y{}, the MAE is {:.3f}, MSE is {:.3f} and RMSE is {:.3f}."
        .format(n, mae_i, mse_i, rmse_i))
    return

def standardize(X):
    X_std=(X - np.mean(X)) / np.std(X)
    return X_std


In [84]:
X_train, X_valid, X_test, y_train, y_valid, y_test=train_valid_test_split(X,y)
w_hat_rid1=w_hat_ridge_calc(X_train, y_train[:, 0],1)
w_hat_rid2=w_hat_ridge_calc(X_train, y_train[:, 1],1)
y_predict_ridge_train1=predict(X_train, w_hat_rid1)
y_predict_ridge_train2=predict(X_train, w_hat_rid2)
print("###TRAINING DATASET RESULTS###")
print("-------------------------")
print("Ridge Regression Results on Training Data:")
print("-------------------------")
compare(y_train[:, 0], y_predict_ridge_train1,1)
compare(y_train[:, 1], y_predict_ridge_train2,2)

print("-------------------------")
print("###VALIDATION DATASET RESULTS###")
# validation dataset calculation and results

w_hat_lin1=w_hat_lin_calc(X_train, y_train[:, 0])
w_hat_lin2=w_hat_lin_calc(X_train, y_train[:, 1])
y_predict_linear1=predict(X_valid, w_hat_lin1)
y_predict_linear2=predict(X_valid, w_hat_lin2)
print("-------------------------")
print("Linear Regression Results using np.linalg.inv:")
print("-------------------------")
compare(y_valid[:, 0], y_predict_linear1,1)
compare(y_valid[:, 1], y_predict_linear2,2)
print("-------------------------")

X_train, X_valid, X_test, y_train, y_valid, y_test=train_valid_test_split(X,y)
w_hat_lin_std1=w_hat_lin_calc(standardize(X_train), y_train[:, 0])
w_hat_lin_std2=w_hat_lin_calc(standardize(X_train), y_train[:, 1])
y_predict_linear1=predict(standardize(X_valid), w_hat_lin_std1)
y_predict_linear2=predict(standardize(X_valid), w_hat_lin_std2)
print("-------------------------")
print("Linear Regression Results using standardized data:")
print("-------------------------")
compare(y_valid[:, 0], y_predict_linear1,1)
compare(y_valid[:, 1], y_predict_linear2,2)
print("-------------------------")

w_hat_lin_calc_pseudo1=w_hat_lin_calc_pseudo(X_train, y_train[:, 0])
w_hat_lin_calc_pseudo2=w_hat_lin_calc_pseudo(X_train, y_train[:, 1])
y_predict_linear1=predict(X_valid, w_hat_lin_calc_pseudo1)
y_predict_linear2=predict(X_valid, w_hat_lin_calc_pseudo2)
print("-------------------------")
print("Linear Regression Results using np.linalg.pinv:")
print("-------------------------")
compare(y_valid[:, 0], y_predict_linear1,1)
compare(y_valid[:, 1], y_predict_linear2,2)
print("-------------------------") 


print("-------------------------")
y_predict_ridge1=predict(X_valid, w_hat_rid1)
y_predict_ridge2=predict(X_valid, w_hat_rid2)

print("Ridge Regression Results of validation data:")
print("-------------------------")
compare(y_valid[:, 0], y_predict_ridge1,1)
compare(y_valid[:, 1], y_predict_ridge2,2)
print("-------------------------")

Data amounts to training data 537, validation data 76 and testing data 153. 
###TRAINING DATASET RESULTS###
-------------------------
Ridge Regression Results on Training Data:
-------------------------
For the Y1, the MAE is 2.125, MSE is 9.064 and RMSE is 3.011.
For the Y2, the MAE is 2.318, MSE is 10.678 and RMSE is 3.268.
-------------------------
###VALIDATION DATASET RESULTS###
-------------------------
Linear Regression Results using np.linalg.inv:
-------------------------
For the Y1, the MAE is 17.212, MSE is 460.190 and RMSE is 21.452.
For the Y2, the MAE is 20.092, MSE is 636.090 and RMSE is 25.221.
-------------------------
Data amounts to training data 537, validation data 76 and testing data 153. 
-------------------------
Linear Regression Results using standardized data:
-------------------------
For the Y1, the MAE is 4.677, MSE is 31.515 and RMSE is 5.614.
For the Y2, the MAE is 5.319, MSE is 40.053 and RMSE is 6.329.
-------------------------
------------------------

In [81]:
print("-------------------------")
print("###TEST DATASET RESULTS###")
# testing dataset calculation and results   
y_predict_linear1=predict(X_test, w_hat_lin1)
y_predict_linear2=predict(X_test, w_hat_lin2)
print("-------------------------")
print("Linear Regression Results using np.linalg.inv for test dataset:")
print("-------------------------")
compare(y_test[:, 0], y_predict_linear1,1)
compare(y_test[:, 1], y_predict_linear2,2)
print("-------------------------")
y_predict_linear1=predict(standardize(X_test), w_hat_lin_std1)
y_predict_linear2=predict(standardize(X_test), w_hat_lin_std2)
print("-------------------------")
print("Linear Regression Results using standardized data for test dataset:")
print("-------------------------")
compare(y_test[:, 0], y_predict_linear1,1)
compare(y_test[:, 1], y_predict_linear2,2)
print("-------------------------")
y_predict_linear1=predict(X_test, w_hat_lin_calc_pseudo1)
y_predict_linear2=predict(X_test, w_hat_lin_calc_pseudo2)
print("-------------------------")
print("Linear Regression Results using np.linalg.pinv for test dataset:")
print("-------------------------")
compare(y_test[:, 0], y_predict_linear1,1)
compare(y_test[:, 1], y_predict_linear2,2)
print("-------------------------")
print("-------------------------")
y_predict_ridge1=predict(X_test, w_hat_rid1)
y_predict_ridge2=predict(X_test, w_hat_rid2)
print("Ridge Regression Results for test dataset:")
print("-------------------------")
compare(y_test[:, 0], y_predict_ridge1,1)
compare(y_test[:, 1], y_predict_ridge2,2)
print("-------------------------")


-------------------------
###TEST DATASET RESULTS###
-------------------------
Linear Regression Results using np.linalg.inv for test dataset:
-------------------------
For the Y1, the MAE is 18.340, MSE is 537.173 and RMSE is 23.177.
For the Y2, the MAE is 21.310, MSE is 744.193 and RMSE is 27.280.
-------------------------
-------------------------
Linear Regression Results using standardized data for test dataset:
-------------------------
For the Y1, the MAE is 6.698, MSE is 64.264 and RMSE is 8.017.
For the Y2, the MAE is 6.875, MSE is 70.735 and RMSE is 8.410.
-------------------------
-------------------------
Linear Regression Results using np.linalg.pinv for test dataset:
-------------------------
For the Y1, the MAE is 2.750, MSE is 10.890 and RMSE is 3.300.
For the Y2, the MAE is 2.585, MSE is 10.836 and RMSE is 3.292.
-------------------------
-------------------------
Ridge Regression Results for test dataset:
-------------------------
For the Y1, the MAE is 2.186, MSE is 

In [82]:
from sklearn.linear_model import LinearRegression, Ridge
X_df=pd.DataFrame(X)
train=int(X.shape[0]*0.7)
valid=int(X.shape[0]*0.1)
test=int(X.shape[0]*0.2)
X_train=X_df.iloc[0:train,:].values
X_valid=X_df.iloc[train:train+valid,:].values
X_test=X_df.iloc[train+valid:,:].values
y_df=pd.DataFrame(y)
y_train=y_df.iloc[0:train,:].values
y_valid=y_df.iloc[train:train+valid,:].values
y_test=y_df.iloc[train+valid:,:].values
LR_model=LinearRegression()
LR_model.fit(X_train, y_train)
y_predict_sklearn=LR_model.predict(X_test)
RR_model=Ridge(alpha=1)
RR_model.fit(X_train, y_train)
y_predict_ridge_sklearn=RR_model.predict(X_test)

print("--------------")
print("Sklearn Linear Regression Results:")
print("--------------")
compare(y_test, y_predict_sklearn,1)

print("--------------")
print("Sklearn Ridge Regression Results:")
print("--------------")
compare(y_test, y_predict_ridge_sklearn,2)
print("-------------------------")

--------------
Sklearn Linear Regression Results:
--------------
For the Y1, the MAE is 2.668, MSE is 10.863 and RMSE is 3.296.
--------------
Sklearn Ridge Regression Results:
--------------
For the Y2, the MAE is 2.309, MSE is 9.354 and RMSE is 3.058.
-------------------------
